# VGG网络设计思想与案例分析
VGG网络是使用重复元素的网络。AlexNet在LeNet的基础上增加了3个卷积层。但AlexNet作者对它们的卷积窗口、输出通道数和构造顺序均做了大量的调整。虽然AlexNet指明了深度卷积神经网络可以取得出色的结果，但并没有提供简单的规则以指导后来的研究者如何设计新的网络。我们将在本章的后续几节里介绍几种不同的深度网络设计思路。它的名字来源于论文作者所在的实验室Visual Geometry Group。VGG提出了可以通过重复使用简单的基础块来构建深度模型的思路。

## VGG块
VGG块的组成规律是：连续使用数个相同的填充为1、窗口形状为3×33×3的卷积层后接上一个步幅为2、窗口形状为2×22×2的最大池化层。卷积层保持输入的高和宽不变，而池化层则对其减半。

使用vgg_block函数来实现这个基础的VGG块，它可以指定卷积层的数量num_convs和输出通道数num_channels。

## VGG网络
与AlexNet和LeNet一样，VGG网络由卷积层模块后接全连接层模块构成。卷积层模块串联数个vgg_block，其超参数由变量conv_arch定义。该变量指定了每个VGG块里卷积层个数和输出通道数。全连接模块则跟AlexNet中的一样。

## 本案例可以构建任意VGG框架
方法：只要修改模型结构参数conv_arch即可，例如

VGG11： conv_arch= ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))

In [112]:
import tensorflow as tf
from tensorflow.keras import layers, models, losses, datasets,optimizers
import numpy as np


print('TensorFlow version:', tf.__version__)
print('Numpy version:',np.__version__)

for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

TensorFlow version: 2.2.0
Numpy version: 1.18.1


In [113]:

class VGG():
    def __init__(self):
        fashion_mnist = datasets.fashion_mnist
        (self.train_images, self.train_labels), (self.test_images, self.test_labels) = fashion_mnist.load_data()
        self.train_images = np.expand_dims(self.train_images.astype(np.float32)/255.0,axis=-1)
        self.test_images = np.expand_dims(self.test_images.astype(np.float32)/255.0,axis=-1)
        self.train_labels = self.train_labels.astype(np.int32)
        self.test_labels = self.test_labels.astype(np.int32)
        self.num_train, self.num_test = self.train_images.shape[0], self.test_images.shape[0]

    def get_batch_train(self, batch_size):
        index = np.random.randint(0, np.shape(self.train_images)[0], batch_size)
        #need to resize images to (224,224)
        resized_images = tf.image.resize_with_pad(self.train_images[index],224,224,)
        return resized_images.numpy(), self.train_labels[index]

    def get_batch_test(self, batch_size):
        index = np.random.randint(0, np.shape(self.test_images)[0], batch_size)
        #need to resize images to (224,224)
        resized_images = tf.image.resize_with_pad(self.test_images[index],224,224,)
        return resized_images.numpy(), self.test_labels[index]
    
    def vgg_block(self, num_convs, num_channels):
        blk = models.Sequential()
        for _ in range(num_convs):
            blk.add(layers.Conv2D(num_channels, kernel_size = 3, padding = 'same', activation = 'relu'))
        blk.add(layers.MaxPool2D(pool_size = 2, strides = 2))
        return blk
    
    def build_vgg(self, conv_arch):
        tf.keras.backend.clear_session()
        net = models.Sequential()
        net.add(layers.InputLayer(input_shape=(224,224,1)))
        for (num_convs, num_channels) in conv_arch:
            net.add(self.vgg_block(num_convs, num_channels))
        net.add(models.Sequential([layers.Flatten(), 
                                   layers.Dense(4096, activation='relu'),
                                   layers.Dropout(0.5),
                                   layers.Dense(4096, activation='relu'), layers.Dropout(0.5),
                                   layers.Dense(10, activation = 'sigmoid')]))
        optimizer = optimizers.SGD(learning_rate=0.05, momentum=0.0, nesterov=False)
        net.compile(optimizer=optimizer,
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        net.summary()
        return net

    def see_output_shape(self, net):
        X = tf.random.uniform((1,224,224,1))
        for blk in net.layers:
            X = blk(X)
            print(blk.name, 'output shape:\t', X.shape)

    def train_vgg(self, net, epoch):
    #   net.load_weights("./ModelTrain/vgg_weights.h5")
        num_iter = self.num_train//batch_size
        with tf.device('/gpu:0'):
            for e in range(epoch):
                for n in range(num_iter):
                    x_batch, y_batch = self.get_batch_train(batch_size)
                    net.fit(x_batch, y_batch)
        
        net.save_weights("./ModelTrain/vgg11_weights.h5")
        return net 


In [ ]:
if __name__ == '__main__':
    
    vgg = VGG()
    batch_size = 128
    x_batch, y_batch = vgg.get_batch_train(batch_size)
    print("x_batch shape:",x_batch.shape,"y_batch shape:", y_batch.shape)


    conv_arch= ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))
    ratio = 4
    small_conv_arch = [(pair[0], pair[1] // ratio) for pair in conv_arch]
    net = vgg.build_vgg(small_conv_arch)
    vgg.see_output_shape(net)

    net = vgg.train_vgg(net, 5)
    ## Accuracy:0.9297